In [6]:
import numpy as np

In [104]:
import math
import numpy as np

class Neuron:

    def __init__(self, n_inputs, learning_rate=0.01):
        
        self.lr = learning_rate
        self.weights = np.random.randn(n_inputs) * self.lr
        self.bias = np.zeros(1)
        self.inputs = None
        # For storing gradients
        self.grad_weights = None
        self.grad_bias = None

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.bias
        return self.output

    def backward(self, grad_output):
        self.grad_inputs = np.dot(grad_output, self.weights.T)      # Gradient with respect to the input
        # Compute gradients of weights and bias
        self.grad_weights = np.dot(self.inputs.T, grad_output)      # Gradient with respect to the weights
        self.grad_bias = np.sum(grad_output, axis=0, keepdims=True) # Gradient with respect to the bias

        # Here, instead of directly updating weights, store gradients for an optimizer to use
        return self.grad_inputs, self.grad_weights, self.grad_bias
    
    def __repr__(self):
        """
        Return a string representation of the neuron.
        """
        return f"Neuron(weight={self.weights}, bias={self.bias}, grad_weights={self.grad_weights}, grad_bias={self.grad_bias}, learning_rate={self.learning_rate})"



class PerceptronFromNeuron:
    def __init__(self, n_inputs, learning_rate=0.01):
        # The perceptron is a single neuron in this context
        self.neuron = Neuron(n_inputs, learning_rate=learning_rate)
    
    def activation(self, x):
        # Step function
        return np.where(x >= 0, 1, 0)

    def forward(self, inputs):
        # Compute the weighted sum (linear output)
        linear_output = np.dot(inputs, self.neuron.weights) + self.neuron.bias
        # Apply the activation function
        return self.activation(linear_output)
        
    def train(self, X, y, epochs):
        for _ in range(epochs):
            for input, label in zip(X, y):
                # Compute the output of the perceptron
                output = self.forward(input)
                # Calculate the error
                error = label - output
                # Update the weights and bias
                self.neuron.weights += self.neuron.learning_rate * error * input
                self.neuron.bias += self.neuron.learning_rate * error
    
    def predict(self, X):
        return self.forward(X)

In [105]:
class Perceptron:

    def __init__(self, input_dim, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        # Initialize weights and bias
        self.weights = np.zeros(input_dim)
        self.bias = 0.0

    def activation(self, x):
        """ Step function """
        return np.where(x >= 0, 1, 0)

    def forward(self, x):
        """Compute the linear output of the perceptron."""
        # Compute the weighted sum (linear output)
        linear_output = np.dot(x, self.weights) + self.bias
        # Apply the activation function
        return self.activation(linear_output)

    def train(self, X, y):
        """Train the perceptron model on the training data."""        
        # Training loop
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                y_pred = self.forward(x_i)
                # Calculate the error
                error = y[idx] - y_pred
                # Update weights and bias
                self.weights += self.lr * error * x_i
                self.bias += self.lr * error
    
    def predict(self, X):
        """Predict the class labels for the input data."""
        return self.forward(X)

In [101]:
# AND function
X_and = np.array([[0,0], [0,1], [1,0], [1,1]])
y_and = np.array([0, 0, 0, 1])

# OR function
X_or = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_or = np.array([0, 1, 1, 1])

print(X_or)
print(y_or)

[[0 0]
 [0 1]
 [1 0]
 [1 1]]
[0 1 1 1]


In [95]:
# Model Hyperparameters

n_iters = 10000      # number of iterations
lr = 0.01           # learning rate

In [99]:
# Training the model on OR function
print("Training on OR function: n_iters = {}, lr = {}".format(n_iters, lr))
custom_or_perceptron = Perceptron(lr=lr, n_iters=n_iters)
custom_or_perceptron.fit(X_or, y_or)
print("Training complete")

Training on OR function: n_iters = 10000, lr = 0.01
Training complete


In [100]:
# Testing
print("Testing the OR perceptron")
predictions = custom_or_perceptron.predict(X_or)
print(predictions)
print("labels       : ", y_or)
print("predictions  : ", predictions.round())
for input in X_or:
    pred_out = custom_or_perceptron.predict(input)
    print(f"{input} -> {pred_out} -> {pred_out.round()}")


Testing the OR perceptron
[0 1 1 1]
labels       :  [0 1 1 1]
predictions  :  [0 1 1 1]
[0 0] -> 0 -> 0
[0 1] -> 1 -> 1
[1 0] -> 1 -> 1
[1 1] -> 1 -> 1


# Perceptron using Pytorch 

In [70]:
import torch

class PerceptronTorch(torch.nn.Module):
    def __init__(self, input_dim):
        super(PerceptronTorch, self).__init__()
        self.fc = torch.nn.Linear(input_dim, 1)
        
    def forward(self, x):
        return torch.sigmoid(self.fc(x))        # sigmoid function as activation function
    
    def predict(self, x):
        with torch.no_grad():
            return self.forward(x)
    
    def fit(self, X, y, lr=0.01, n_iters=1000, optimizer=None, loss_criterion=None):
        if optimizer is None:
            optimizer = torch.optim.SGD(self.parameters(), lr=lr)
        if loss_criterion is None:
            loss_criterion = torch.nn.BCELoss()

        for _ in range(n_iters):
            optimizer.zero_grad()
            output = self.forward(X)
            loss = loss_criterion(output, y)
            loss.backward()
            optimizer.step()


In [72]:
def create_optimizer(model, name="sgd", lr=0.01):
    if name == "sgd":
        return torch.optim.SGD(model.parameters(), lr=lr)           # Stochastic Gradient Descent
    elif name == "adam":
        return torch.optim.Adam(model.parameters(), lr=lr)          # Adam optimizer
    elif name == "rmsprop":
        return torch.optim.RMSprop(model.parameters(), lr=lr)       # RMSprop optimizer
    elif name == "adagrad":
        return torch.optim.Adagrad(model.parameters(), lr=lr)       # Adagrad optimizer
    elif name == "adadelta":
        return torch.optim.Adadelta(model.parameters(), lr=lr)      # Adadelta optimizer
    elif name == "adamw":
        return torch.optim.AdamW(model.parameters(), lr=lr)         # AdamW optimizer
    elif name == "adamax":
        return torch.optim.Adamax(model.parameters(), lr=lr)        # Adamax optimizer
    elif name == "asgd":
        return torch.optim.ASGD(model.parameters(), lr=lr)          # ASGD optimizer
    elif name == "rprop":
        return torch.optim.Rprop(model.parameters(), lr=lr)         # Rprop optimizer
    elif name == "lbfgs":
        return torch.optim.LBFGS(model.parameters(), lr=lr)         # LBFGS optimizer
    else:
        return torch.optim.SGD(model.parameters(), lr=lr)           # Stochastic Gradient Descent


In [73]:
# loss_criterion = torch.nn.BCELoss()              # Binary Cross-Entropy Loss for binary classification
# loss_criterion = torch.nn.MSELoss()            # Mean Squared Error Loss for regression
# loss_criterion = torch.nn.CrossEntropyLoss()   # Cross-Entropy Loss for multi-class classification

In [ ]:
# OR function data
X_or = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_or = torch.tensor([0, 1, 1, 1], dtype=torch.float32).view(-1, 1)

# AND function data
X_and = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_and = torch.tensor([0, 0, 0, 1], dtype=torch.float32).view(-1, 1)

print(X_or)
print(y_or)

In [87]:
# Model Hyperparameters

n_iters = 10000      # number of iterations
lr = 0.01           # learning rate

In [88]:
# Training the model on OR function
print("Training on OR function: n_iters = {}, lr = {}".format(n_iters, lr))
or_perceptron = PerceptronTorch(input_dim=2)
or_optimizer = create_optimizer(or_perceptron, name="sgd", lr=0.01)
loss_criterion = torch.nn.BCELoss()
or_perceptron.fit(X_or, y_or, lr, n_iters, optimizer=or_optimizer, loss_criterion=loss_criterion)
print("Training complete")

Training on OR function: n_iters = 10000, lr = 0.01
Training complete


In [89]:
# Testing 
print("Testing the OR perceptron:")
predictions = or_perceptron.predict(X_or)
print(predictions)
print("labels       : ", y_or.view(-1))
print("predictions  : ", predictions.round().view(-1))
for input in X_or:
    pred_out = or_perceptron.predict(input)
    print(f"{input} -> {pred_out} -> {pred_out.round().view(-1).numpy()}")

Testing trained OR perceptron:
tensor([[0.1841],
        [0.9262],
        [0.9308],
        [0.9987]])
labels       :  tensor([0., 1., 1., 1.])
predictions  :  tensor([0., 1., 1., 1.])
tensor([0., 0.]) -> tensor([0.1841]) -> [0.]
tensor([0., 1.]) -> tensor([0.9262]) -> [1.]
tensor([1., 0.]) -> tensor([0.9308]) -> [1.]
tensor([1., 1.]) -> tensor([0.9987]) -> [1.]


In [90]:
# Training the model on AND function
print("Training on AND function:")
and_perceptron = PerceptronTorch(input_dim=2)
and_optimizer = create_optimizer(and_perceptron, name="sgd", lr=0.01)
loss_criterion = torch.nn.BCELoss()
and_perceptron.fit(X_and, y_and, lr, n_iters, optimizer=and_optimizer, loss_criterion=loss_criterion)
print("Training complete")

Training on AND function:
Training complete


In [91]:
# Testing
print("Testing the AND perceptron:")
predictions = and_perceptron.predict(X_and)
print(predictions)
print("labels       : ", y_and.view(-1))
print("predictions  : ", predictions.round().view(-1))
for input in X_and:
    pred_out = and_perceptron.predict(input)
    print(f"{input} -> {pred_out} -> {pred_out.round().view(-1).numpy()}")

Testing trained perceptron:
tensor([[0.0076],
        [0.1469],
        [0.1450],
        [0.7919]])
labels       :  tensor([0., 0., 0., 1.])
predictions  :  tensor([0., 0., 0., 1.])
tensor([0., 0.]) -> tensor([0.0076]) -> [0.]
tensor([0., 1.]) -> tensor([0.1469]) -> [0.]
tensor([1., 0.]) -> tensor([0.1450]) -> [0.]
tensor([1., 1.]) -> tensor([0.7919]) -> [1.]
